In [ ]:
import os
import sys
import fiona 
import csv
import time
from fiona.crs import from_epsg
import pandas as pd # pip install necessary
import overpy # pip install necessary
import geopandas # pip install geopandas
import numpy as np

pd.set_option('display.max_rows', 500)

In [ ]:
#create list of 400 regionalschluessel (ARS) of all Landkreise/kreisfreie Städte/Stadtkreise/Stadtstaaten

#read file from Statistisches Bundesamt
df_regionalschluessel = pd.read_excel('../Infrastructure/regionalschluessel.xlsx', skiprows=6, sheet_name='Onlineprodukt_Gemeinden31122023')

#remove columns that dont contain ARS
df_regionalschluessel= df_regionalschluessel.drop(df_regionalschluessel.columns[[7,8,9,10,11,12,13,14,15,16,17,18,19]], axis=1)

#rename columns
df_regionalschluessel = df_regionalschluessel.rename(columns={"10": "satzart","Unnamed: 1": "textkennzeichen", "01": "a", "Unnamed: 3": "b", "Unnamed: 4": "c"})

#only keep rows with first column ("Satzart") =40 or =41 (only keeps Landkreise etc., throws out municipalities)
df_regionalschluessel = df_regionalschluessel[df_regionalschluessel['satzart']=="40"]

#drop all columns not part of the ARS (besides textkennzeichen, still needed for leading zeros)
df_regionalschluessel= df_regionalschluessel.drop(df_regionalschluessel.columns[[0,5,6]], axis=1)

df_regionalschluessel

In [ ]:
#convert to int and str
df_regionalschluessel['a'] = df_regionalschluessel['a'].astype(int).astype(str)
df_regionalschluessel['b'] = df_regionalschluessel['b'].astype(int).astype(str)
df_regionalschluessel['c'] = df_regionalschluessel['c'].astype(int).astype(str)

#add leading zeros to a and c
df_regionalschluessel['a'] = df_regionalschluessel['a'].apply(lambda x: '0' + x if len(x) == 1 else x)
df_regionalschluessel['c'] = df_regionalschluessel['c'].apply(lambda x: '0' + x if len(x) == 1 else x)

#create 5-digit-ARS from 3 columns
df_regionalschluessel['ARS'] = df_regionalschluessel.a + df_regionalschluessel.b + df_regionalschluessel.c

#drop a, b and c
df_regionalschluessel = df_regionalschluessel.drop(df_regionalschluessel.columns[[1,2,3]], axis=1)

#add trailing zeros for Kreisfreie Städte or Stadtkreise (BaWü)
df_regionalschluessel['ARS'] = df_regionalschluessel.apply(lambda row: row['ARS'] + '0000000' if (row['textkennzeichen'] == 41.0 or row['textkennzeichen'] == 42.0) else row['ARS'], axis=1)

#create list
list_regionalschluessel = df_regionalschluessel['ARS'].tolist()

list_regionalschluessel

In [ ]:
#create empty dataframe for the length of the bicycle and road networks
network_lengths = pd.DataFrame(data={'ars': [], 'cycling': [], 'roads': []})
network_lengths

In [ ]:
#D O W N L O A D   (cycling)
#ca. 20 min

#the code for downloading the network data and calculating its length was written referencing Tobias Link's (KOSIS-Gemeinschaft Urban Audit, c/o Stadt Mannheim) similar work for the Urban Audit: https://www.staedtestatistik.de/arbeitsgemeinschaften/kosis/urban-audit 

# create tick
tick = 0

# download OSM __cycling__ data for every Landkreis/kreisfreie Stadt/Stadtstaat
for ars in list_regionalschluessel:
   
    # increase tick for visualisation of progress
    tick += 1
    
    api = overpy.Overpass(url="https://overpass-api.de/api/interpreter")
    
    cyclingnetwork = api.query(f"""
        [timeout:900]
        [bbox:47.1119,5.4021,55.1888,15.3337]
        ;
        area["de:regionalschluessel" = "{ars}"]->.searchArea;
        (
        way["bicycle"="designated"](area.searchArea);
        way["bicycle_road"="yes"](area.searchArea);
        way["bicycle:lanes"="yes"](area.searchArea);
        way["cyclestreet"="yes"](area.searchArea);
        way["cycleway"="cyclestreet"](area.searchArea);
        way["cycleway"="lane"](area.searchArea);
        way["cycleway:left"="lane"](area.searchArea);
        way["cycleway:right"="lane"](area.searchArea);
        way["cycleway"="track"](area.searchArea);
        way["cycleway"="opposite_track"](area.searchArea);
        way["cycleway:right"="track"](area.searchArea);
        way["cycleway:left"="track"](area.searchArea);
        way["highway"="cycleway"](area.searchArea);
    );
    (
        ._;
        >;
    );
    out;
    """)
    
    # save network to local folder
    schema = {'geometry': 'LineString', 'properties': {'Name':'str:80'}}
    
    #cycleway_OSM defines the location of the respective shape file
    cycleway_OSM = '../Infrastructure/networks/cycling/' + ars + '.shp'
     
    with fiona.open(cycleway_OSM, 'w',crs=from_epsg(4326),driver='ESRI Shapefile', schema=schema, encoding='utf-8') as output:
        for way in cyclingnetwork.ways:
            # the shapefile geometry use (lon,lat) 
            feature = {'type': 'LineString', 'coordinates':[(node.lon, node.lat) for node in way.nodes]}
            prop = {'Name': way.tags.get("name", "n/a")}
            output.write({'geometry': feature, 'properties':prop})
                
    print("Daten wurden für " + ars + " heruntergeladen")
     # show progress
    print(tick, "/", len(list_regionalschluessel))
    print("")

In [ ]:
#D O W N L O A D   (roads) (without Berlin)
#ca. 60 min

# create tick
tick = 0

#remove Berlin from the list, will be downloaded separately
list_regionalschluessel_woBE = list_regionalschluessel
list_regionalschluessel_woBE.remove("110000000000")

# download OSM __road__ data for every Landkreis/kreisfreie Stadt/Stadtstaat
for ars in list_regionalschluessel_woBE:
   
    # increase tick for visualisation of progress
    tick += 1
    
    api = overpy.Overpass(url="https://overpass-api.de/api/interpreter")
    
    cyclingnetwork = api.query(f"""
        [timeout:9000]
        [bbox:47.1119,5.4021,55.1888,15.3337]
        ;
        area["de:regionalschluessel" = "{ars}"]->.searchArea;
        (
        way["highway"="unclassified"](area.searchArea);
        way["highway"="residential"](area.searchArea);
        way["highway"="living_street"](area.searchArea);
        way["highway"="tertiary"](area.searchArea);
        way["highway"="secondary"](area.searchArea);
        way["highway"="primary"](area.searchArea);
    );
    (
        ._;
        >;
    );
    out;
    """)
    
    # save network to local folder
    schema = {'geometry': 'LineString', 'properties': {'Name':'str:80'}}
    
    #roads_OSM defines the location of the respective shape file
    roads_OSM = '../Infrastructure/networks/roads/' + ars + '.shp'
     
    with fiona.open(roads_OSM, 'w',crs=from_epsg(4326),driver='ESRI Shapefile', schema=schema, encoding='utf-8') as output:
        for way in cyclingnetwork.ways:
            # the shapefile geometry use (lon,lat) 
            feature = {'type': 'LineString', 'coordinates':[(node.lon, node.lat) for node in way.nodes]}
            prop = {'Name': way.tags.get("name", "n/a")}
            output.write({'geometry': feature, 'properties':prop})
                
    print("Daten wurden für " + ars + " heruntergeladen")
     # show progress
    print(tick, "/", len(list_regionalschluessel))
    print("")

In [ ]:
#D O W N L O A D   (roads) (Berlin only)
#ca. 60 min

# create tick
tick = 0

list_regionalschluessel_BEonly=["110000000000"]

# download OSM __road__ data
for ars in list_regionalschluessel_BEonly:
   
    # increase tick for visualisation of progress
    tick += 1
    
    api = overpy.Overpass(url="https://overpass-api.de/api/interpreter")
    
    cyclingnetwork = api.query(f"""
        [timeout:9000]
        [bbox:47.1119,5.4021,55.1888,15.3337]
        ;
        area["de:regionalschluessel" = "{ars}"]->.searchArea;
        (
        way["highway"="unclassified"](area.searchArea);
        way["highway"="residential"](area.searchArea);
        way["highway"="living_street"](area.searchArea);
        way["highway"="tertiary"](area.searchArea);
        way["highway"="secondary"](area.searchArea);
        way["highway"="primary"](area.searchArea);
    );
    (
        ._;
        >;
    );
    out;
    """)
    
    # save network to local folder
    schema = {'geometry': 'LineString', 'properties': {'Name':'str:80'}}
    
    #roads_OSM defines the location of the respective shape file
    roads_OSM = '../Infrastructure/networks/roads/' + ars + '.shp'
     
    with fiona.open(roads_OSM, 'w',crs=from_epsg(4326),driver='ESRI Shapefile', schema=schema, encoding='utf-8') as output:
        for way in cyclingnetwork.ways:
            # the shapefile geometry use (lon,lat) 
            feature = {'type': 'LineString', 'coordinates':[(node.lon, node.lat) for node in way.nodes]}
            prop = {'Name': way.tags.get("name", "n/a")}
            output.write({'geometry': feature, 'properties':prop})
                
    print("Daten wurden für " + ars + " heruntergeladen")
     # show progress
    print(tick, "/", len(list_regionalschluessel))
    print("")

In [ ]:
# C A L C U L A T I O N   (cycling)
#ca. 2 min

# create tick
tick = 0

for ars in list_regionalschluessel:
   
    # increase tick for visualisation of progress
    tick += 1
    
    #cycleway_OSM defines the location of the respective shape file
    cycleway_OSM = '../Infrastructure/networks/cycling/' + ars + '.shp'
    
    # open the saved shapefile
    cyclingnetwork = geopandas.read_file(cycleway_OSM, driver='Shapefile')
    
    cyclingnetwork = cyclingnetwork.to_crs('EPSG:25832')
    
    # calculate length of cycling network and save into list with respective ars
    cycling_length = sum(cyclingnetwork.length)/1000
    new_row = {'ars': ars, 'cycling': cycling_length, 'roads': None}
    network_lengths = pd.concat([network_lengths, pd.DataFrame([new_row])], ignore_index=True)   
    
    print("Daten wurden für " + ars + " wurden ausgewertet")
    # show progress
    print(tick, "/", len(list_regionalschluessel))
    print("")
    
network_lengths

In [ ]:
# C A L C U L A T I O N   (roads)
#ca. 10 min

# create tick
tick = 0

for ars in list_regionalschluessel:
   
    # increase tick for visualisation of progress
    tick += 1
    
    #cycleway_OSM defines the location of the respective shape file
    roads_OSM = '../Infrastructure/networks/roads/' + ars + '.shp'
    
    # open the saved shapefile
    roadnetwork = geopandas.read_file(roads_OSM, driver='Shapefile')
    
    roadnetwork = roadnetwork.to_crs('EPSG:25832')
    
    # calculate length of cycling network and save into list with respective ars
    roads_length = sum(roadnetwork.length)/1000

    network_lengths.loc[network_lengths['ars'] == ars, 'roads'] = roads_length
    
    print("Daten wurden für " + ars + " wurden ausgewertet")
    # show progress
    print(tick, "/", len(list_regionalschluessel))
    print("")
    
network_lengths

In [ ]:
#compute cycling infra coverage
network_lengths['cycling_coverage']=network_lengths['cycling'] / network_lengths['roads']

In [ ]:
network_lengths

In [ ]:
network_lengths.to_csv('../Infrastructure/lengths.csv')

In [ ]:
### Write cycling coverage to 1km cells ###

In [ ]:
#by reading the csv, leading zeros are chopped off already!
network_lengths = pd.read_csv('../Infrastructure/lengths.csv')

# bring ars back to 4/5 digits without following zeros to match ags in cells-dataframe
network_lengths['ars_short'] = network_lengths['ars'].astype(str).apply(lambda x: x[:-7] if len(x) > 6 else x).astype(int)
network_lengths

In [ ]:
cells = pd.read_csv('C:/Users/arning/Desktop/11_Mode_Choice/1km_Raumtyp_Slope_ptqual.csv', header=0)

#compute ars of Landkreis from ags of municipality
cells['ars'] = cells['ags'].apply(lambda x: int(str(x)[:-3]) if x >= 1000 else 0)
cells

In [ ]:
#write coverage of cycling infrastructure to 1km cells
# Create a dictionary mapping ars to cycling_coverage
ars_to_cyclingcoverage = dict(zip(network_lengths['ars_short'], network_lengths['cycling_coverage']))

# Use the map function to populate the cycling coverage column in cells
cells['cycling_coverage'] = cells['ars'].map(ars_to_cyclingcoverage)

cells

In [ ]:
#save file
cells.to_csv('../1km_Raumtyp_Slope_ptqual_infra.csv')